In [1]:
import torch
import timm
from collections import OrderedDict

In [2]:
# 아키텍처 정의
model = timm.create_model(
    'resnest101e',
    pretrained=False,
    num_classes=4,
    drop_path_rate=0.2
)

# 가중치 로드 및 Key 수정
model_path = '../data/models/Dist_Model_ver1/final_soup_model.pth'
state_dict = torch.load(model_path, map_location='cpu')

# 'model.' 접두어 제거
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k.replace("model.", "") # 'model.layer1...' -> 'layer1...'
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)
model.eval() # 추론 모드 설정 

# ONNX Export
dummy_input = torch.randn(1, 3, 450, 650) # 학습 시 이미지 사이즈
onnx_path = '../data/models/Dist_Model_ver1/final_model.onnx'

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    export_params=True,
    opset_version=15,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)

print(f'ONNX model saved to: {onnx_path}')

/tmp/ipykernel_11006/1341935790.py:26: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


ONNX model saved to: ../data/models/Dist_Model_ver1/final_model.onnx
